In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import dateutil
import statsmodels
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
sns.set()
pd.options.display.max_columns = None

In [2]:
claims_costs_path = '../Dataset/Modified_claims/'
ehr_path = '../Dataset/ehr/'

In [3]:
ehr = {}
ehr_table_names = os.listdir(ehr_path)
for x in ehr_table_names:
    ehr[x[:-4]] = pd.read_csv(ehr_path+x,encoding = 'latin',low_memory=False)
ehr.keys()

dict_keys(['admissions', 'caregivers', 'cptevents', 'diagnoses_icd', 'drgcodes', 'd_cpt', 'd_icd_diagnoses', 'd_icd_procedures', 'd_items', 'd_labitems', 'icustays', 'inputevents_cv', 'inputevents_mv', 'labevents', 'microbiologyevents', 'patients', 'prescriptions', 'procedureevents_mv', 'procedures_icd', 'services', 'transfers'])

In [4]:
claims_costs = pd.read_csv(claims_costs_path+'claims_to_ehr.csv')
claims_costs.head()

,DESYNPUF_ID,AGE,LENGTH_OF_STAY,GENDER,RACE,AGE_BINS,LENGTH_OF_STAY_BINS,TOTAL_COST,PAYABLE,RATIO,GAP
0,1EADDE572193B956,67,4,Female,WHITE,"(62.5, 70.2]","(3.5, 7.0]",5024.0,1024.0,0.203822,4000.0
1,216C2BA9AB57CC61,75,4,Male,WHITE,"(70.2, 77.9]","(3.5, 7.0]",5024.0,1024.0,0.203822,4000.0
2,22B5C5BC772C32A5,77,2,Male,WHITE,"(70.2, 77.9]","(-0.035, 3.5]",12068.0,1068.0,0.088499,11000.0
3,27584D29C341FB19,67,3,Female,WHITE,"(62.5, 70.2]","(-0.035, 3.5]",5068.0,1068.0,0.210734,4000.0
4,28314AB0026522E2,70,0,Male,WHITE,"(62.5, 70.2]","(-0.035, 3.5]",7148.0,1068.0,0.149412,6080.0


In [5]:
pd.merge(ehr['admissions'],ehr['patients'],how='left',on=['subject_id'])

,row_id_x,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,row_id_y,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1,19.0,F,2131-05-07,NaN,NaN,NaN,0.0
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,20.0,M,2082-07-17,NaN,NaN,NaN,0.0
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1,20.0,M,2082-07-17,NaN,NaN,NaN,0.0
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1,21.0,M,2100-05-31,NaN,NaN,NaN,0.0
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1,22.0,M,2101-11-21,NaN,NaN,NaN,0.0
5,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,V-TACH,0,1,23.0,M,2054-05-04,2128-02-25,NaN,2128-02-25,1.0
6,27,27,134931,2191-11-30 22:16:00,2191-12-03 14:45:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1,24.0,F,2191-11-30,NaN,NaN,NaN,0.0
7,28,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,25.0,M,2103-04-15,NaN,NaN,NaN,0.0
8,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,NaN,NaN,UNSTABLE ANGINA\CATH,0,1,26.0,M,1872-10-14,NaN,NaN,NaN,0.0
9,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,STATUS EPILEPTICUS,1,1,27.0,M,2036-05-17,2108-08-30,2108-08-30,2108-08-30,1.0


In [6]:
admi = ehr['admissions']
admi.dropna(inplace=True,subset=["diagnosis"])
admi

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
5,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,V-TACH,0,1
6,27,27,134931,2191-11-30 22:16:00,2191-12-03 14:45:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1
7,28,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
8,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,NaN,NaN,UNSTABLE ANGINA\CATH,0,1
9,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,STATUS EPILEPTICUS,1,1


In [7]:
searchfor = ["ANGINA","MYOCARDIAL","CORONARY ARTERY","ISCHEMIC HEART","ISCHEMIC HEART"]
columns_to_drop = ["row_id","deathtime","edregtime","edouttime","hospital_expire_flag","has_chartevents_data"]
df_ischemic = pd.merge(admi[admi.diagnosis.str.contains('|'.join(searchfor))].drop(columns = columns_to_drop),ehr['patients'],on='subject_id',how='left').drop(columns=["dod","dod_hosp","dod_ssn"])
df_ischemic.dropna(inplace=True,subset=["dob"])
df_ischemic

,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,diagnosis,row_id,gender,dob,expire_flag
0,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,20.0,M,2082-07-17,0.0
1,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,21.0,M,2100-05-31,0.0
2,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,25.0,M,2103-04-15,0.0
3,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,UNSTABLE ANGINA\CATH,26.0,M,1872-10-14,0.0
4,35,166707,2122-02-10 11:15:00,2122-02-20 15:30:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,AORTIC VALVE DISEASE\CORONARY ARTERY BYPASS GR...,31.0,F,2052-09-01,0.0
5,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,32.0,M,2061-08-17,0.0
6,38,185910,2166-08-10 00:28:00,2166-09-04 11:30:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,ACUTE MYOCARDIAL INFARCTION-SEPSIS,34.0,M,2090-08-31,0.0
7,359,144265,2176-06-30 03:00:00,2176-07-03 15:12:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Self Pay,VIET,BUDDHIST,MARRIED,UNKNOWN/NOT SPECIFIED,R/O MYOCARDIAL INFARCTION,338.0,M,2127-08-18,0.0
8,362,142749,2112-07-10 02:31:00,2112-07-16 13:45:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,UNOBTAINABLE,WIDOWED,WHITE,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,341.0,F,2026-09-23,1.0
9,365,139873,2147-12-15 17:10:00,2147-12-23 12:17:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,JEWISH,MARRIED,WHITE,CORONARY ARTERY DISEASE\CARDIAC CATH,344.0,M,2088-02-23,1.0


In [8]:
def days_to_years( number_of_days ):
    res = int(number_of_days / 365)
    if res>0.0:
        return res
    else:
        return np.NAN
df_ischemic.admittime = pd.to_datetime(df_ischemic.admittime)
df_ischemic.dischtime = pd.to_datetime(df_ischemic.dischtime)
df_ischemic.dob = pd.to_datetime(df_ischemic.dob)
#df_ischemic["AGE"] = (df_ischemic.CLM_ADMSN_DT - df_ischemic.BENE_BIRTH_DT).dt.days.apply(days_to_years)
df_ischemic["LENGTH_OF_STAY"] = (df_ischemic.dischtime - df_ischemic.admittime ).dt.days
df_ischemic["AGE"] = (df_ischemic.admittime - df_ischemic.dob ).dt.days.apply(days_to_years)
df_ischemic.drop(columns=["dob"],inplace=True)
df_ischemic.dropna(inplace=True,subset=["AGE"])
df_ischemic

,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,diagnosis,row_id,gender,expire_flag,LENGTH_OF_STAY,AGE
0,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,20.0,M,0.0,5,71.0
1,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,21.0,M,0.0,2,39.0
2,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,25.0,M,0.0,5,74.0
4,35,166707,2122-02-10 11:15:00,2122-02-20 15:30:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,AORTIC VALVE DISEASE\CORONARY ARTERY BYPASS GR...,31.0,F,0.0,10,69.0
5,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,32.0,M,0.0,8,69.0
6,38,185910,2166-08-10 00:28:00,2166-09-04 11:30:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,ACUTE MYOCARDIAL INFARCTION-SEPSIS,34.0,M,0.0,25,75.0
7,359,144265,2176-06-30 03:00:00,2176-07-03 15:12:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Self Pay,VIET,BUDDHIST,MARRIED,UNKNOWN/NOT SPECIFIED,R/O MYOCARDIAL INFARCTION,338.0,M,0.0,3,48.0
8,362,142749,2112-07-10 02:31:00,2112-07-16 13:45:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,UNOBTAINABLE,WIDOWED,WHITE,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,341.0,F,1.0,6,85.0
9,365,139873,2147-12-15 17:10:00,2147-12-23 12:17:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,JEWISH,MARRIED,WHITE,CORONARY ARTERY DISEASE\CARDIAC CATH,344.0,M,1.0,7,59.0
10,44,181750,2192-11-19 18:14:00,2192-11-27 14:35:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,CORONARY ARTERY DISEASE\CATH,39.0,M,1.0,7,80.0


In [9]:
def to_gender(val):
    if val.gender=="M":
        return "Male"
    else:
        return "Female"
df_ischemic.gender = df_ischemic.apply(to_gender,axis=1)

In [10]:
def to_race(val):
    if val=="WHITE":
        return "WHITE"
    elif "BLACK" in val:
        return "BLACK"
    elif "HISPANIC" in val:
        return "HISPANIC"
    else:
        return "OTHERS"
df_ischemic.ethnicity = df_ischemic.ethnicity.apply(to_race)

In [11]:
claims_costs.head()

,DESYNPUF_ID,AGE,LENGTH_OF_STAY,GENDER,RACE,AGE_BINS,LENGTH_OF_STAY_BINS,TOTAL_COST,PAYABLE,RATIO,GAP
0,1EADDE572193B956,67,4,Female,WHITE,"(62.5, 70.2]","(3.5, 7.0]",5024.0,1024.0,0.203822,4000.0
1,216C2BA9AB57CC61,75,4,Male,WHITE,"(70.2, 77.9]","(3.5, 7.0]",5024.0,1024.0,0.203822,4000.0
2,22B5C5BC772C32A5,77,2,Male,WHITE,"(70.2, 77.9]","(-0.035, 3.5]",12068.0,1068.0,0.088499,11000.0
3,27584D29C341FB19,67,3,Female,WHITE,"(62.5, 70.2]","(-0.035, 3.5]",5068.0,1068.0,0.210734,4000.0
4,28314AB0026522E2,70,0,Male,WHITE,"(62.5, 70.2]","(-0.035, 3.5]",7148.0,1068.0,0.149412,6080.0


In [12]:
_,age_bins = pd.cut(claims_costs.AGE,bins=10,retbins=True)
_,length_bins = pd.cut(claims_costs.LENGTH_OF_STAY,bins=10,retbins=True)
df_ischemic["AGE_BINS"] = pd.cut(df_ischemic.AGE,bins=age_bins)
df_ischemic["LENGTH_OF_STAY_BINS"] = pd.cut(df_ischemic.LENGTH_OF_STAY,bins=length_bins)
df_ischemic.dropna(inplace=True,subset=["AGE_BINS","LENGTH_OF_STAY_BINS"])
df_ischemic

,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,diagnosis,row_id,gender,expire_flag,LENGTH_OF_STAY,AGE,AGE_BINS,LENGTH_OF_STAY_BINS
0,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,20.0,Male,0.0,5,71.0,"(70.2, 77.9]","(3.5, 7.0]"
1,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,21.0,Male,0.0,2,39.0,"(31.7, 39.4]","(-0.035, 3.5]"
2,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,25.0,Male,0.0,5,74.0,"(70.2, 77.9]","(3.5, 7.0]"
4,35,166707,2122-02-10 11:15:00,2122-02-20 15:30:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,AORTIC VALVE DISEASE\CORONARY ARTERY BYPASS GR...,31.0,Female,0.0,10,69.0,"(62.5, 70.2]","(7.0, 10.5]"
5,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,32.0,Male,0.0,8,69.0,"(62.5, 70.2]","(7.0, 10.5]"
6,38,185910,2166-08-10 00:28:00,2166-09-04 11:30:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,ACUTE MYOCARDIAL INFARCTION-SEPSIS,34.0,Male,0.0,25,75.0,"(70.2, 77.9]","(24.5, 28.0]"
7,359,144265,2176-06-30 03:00:00,2176-07-03 15:12:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Self Pay,VIET,BUDDHIST,MARRIED,OTHERS,R/O MYOCARDIAL INFARCTION,338.0,Male,0.0,3,48.0,"(47.1, 54.8]","(-0.035, 3.5]"
8,362,142749,2112-07-10 02:31:00,2112-07-16 13:45:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,UNOBTAINABLE,WIDOWED,WHITE,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,341.0,Female,1.0,6,85.0,"(77.9, 85.6]","(3.5, 7.0]"
9,365,139873,2147-12-15 17:10:00,2147-12-23 12:17:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,JEWISH,MARRIED,WHITE,CORONARY ARTERY DISEASE\CARDIAC CATH,344.0,Male,1.0,7,59.0,"(54.8, 62.5]","(3.5, 7.0]"
10,44,181750,2192-11-19 18:14:00,2192-11-27 14:35:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,MARRIED,OTHERS,CORONARY ARTERY DISEASE\CATH,39.0,Male,1.0,7,80.0,"(77.9, 85.6]","(3.5, 7.0]"


In [13]:
def append_cost(row):
    '''
    Appends the cost to the individual patients in the ehr records based on the 4 parameters(Age,length of stay,race and gender)
    The best possible match will be found and a distribution of the matching parameters will be pulled from the claims database.
    A random sample will be pulled from this distribution and assigned to the row
    '''
    df = claims_costs[(claims_costs.LENGTH_OF_STAY_BINS == str(row.LENGTH_OF_STAY_BINS)) & (claims_costs.AGE_BINS == str(row.AGE_BINS)) & (claims_costs.GENDER == row.gender) & (claims_costs.RACE == row.ethnicity)][["TOTAL_COST","PAYABLE","GAP"]]
    if(df.TOTAL_COST.count()>=1):
        x = df.sample(1)
        return str(str(x.TOTAL_COST.iloc[0])+" "+str(x.PAYABLE.iloc[0]))
    else:
        df = claims_costs[(claims_costs.LENGTH_OF_STAY_BINS == str(row.LENGTH_OF_STAY_BINS)) & (claims_costs.AGE_BINS == str(row.AGE_BINS)) & (claims_costs.GENDER == row.gender)][["TOTAL_COST","PAYABLE","GAP"]]
        if(df.TOTAL_COST.count()>=1):
            x = df.sample(1)
            return str(str(x.TOTAL_COST.iloc[0])+" "+str(x.PAYABLE.iloc[0]))
        else:
            df = claims_costs[(claims_costs.LENGTH_OF_STAY_BINS == str(row.LENGTH_OF_STAY_BINS)) & (claims_costs.AGE_BINS == str(row.AGE_BINS))][["TOTAL_COST","PAYABLE","GAP"]]
            if(df.TOTAL_COST.count()>=1):
                x = df.sample(1)
                return str(str(x.TOTAL_COST.iloc[0])+" "+str(x.PAYABLE.iloc[0]))
            else:
                return np.NAN
df_ischemic["COSTS"] = df_ischemic[["AGE_BINS","LENGTH_OF_STAY_BINS","gender","ethnicity"]].apply(append_cost,axis=1)
df_ischemic["TOTAL_COST"],df_ischemic["PAYABLE"] = df_ischemic["COSTS"].str.split(' ', 1).str
df_ischemic.drop(columns=["COSTS"],inplace=True)
df_ischemic.dropna(inplace=True,subset=["TOTAL_COST","PAYABLE"])
df_ischemic.TOTAL_COST = df_ischemic.TOTAL_COST.astype('float')
df_ischemic.PAYABLE = df_ischemic.PAYABLE.astype('float')
df_ischemic

,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,diagnosis,row_id,gender,expire_flag,LENGTH_OF_STAY,AGE,AGE_BINS,LENGTH_OF_STAY_BINS,TOTAL_COST,PAYABLE
0,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,20.0,Male,0.0,5,71.0,"(70.2, 77.9]","(3.5, 7.0]",12468.0,1068.0
1,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,21.0,Male,0.0,2,39.0,"(31.7, 39.4]","(-0.035, 3.5]",6024.0,1024.0
2,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,25.0,Male,0.0,5,74.0,"(70.2, 77.9]","(3.5, 7.0]",52024.0,1024.0
4,35,166707,2122-02-10 11:15:00,2122-02-20 15:30:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,AORTIC VALVE DISEASE\CORONARY ARTERY BYPASS GR...,31.0,Female,0.0,10,69.0,"(62.5, 70.2]","(7.0, 10.5]",6024.0,1024.0
5,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,32.0,Male,0.0,8,69.0,"(62.5, 70.2]","(7.0, 10.5]",9824.0,1024.0
6,38,185910,2166-08-10 00:28:00,2166-09-04 11:30:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,ACUTE MYOCARDIAL INFARCTION-SEPSIS,34.0,Male,0.0,25,75.0,"(70.2, 77.9]","(24.5, 28.0]",30068.0,1068.0
7,359,144265,2176-06-30 03:00:00,2176-07-03 15:12:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Self Pay,VIET,BUDDHIST,MARRIED,OTHERS,R/O MYOCARDIAL INFARCTION,338.0,Male,0.0,3,48.0,"(47.1, 54.8]","(-0.035, 3.5]",5024.0,1024.0
8,362,142749,2112-07-10 02:31:00,2112-07-16 13:45:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,UNOBTAINABLE,WIDOWED,WHITE,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,341.0,Female,1.0,6,85.0,"(77.9, 85.6]","(3.5, 7.0]",4024.0,1024.0
9,365,139873,2147-12-15 17:10:00,2147-12-23 12:17:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,JEWISH,MARRIED,WHITE,CORONARY ARTERY DISEASE\CARDIAC CATH,344.0,Male,1.0,7,59.0,"(54.8, 62.5]","(3.5, 7.0]",1068.0,1068.0
10,44,181750,2192-11-19 18:14:00,2192-11-27 14:35:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,MARRIED,OTHERS,CORONARY ARTERY DISEASE\CATH,39.0,Male,1.0,7,80.0,"(77.9, 85.6]","(3.5, 7.0]",6068.0,1068.0


Adding total length of stay in icu as a feature

In [14]:
sub_ids = df_ischemic.subject_id.unique()
icustays = ehr['icustays'][ehr['icustays'].subject_id.isin(sub_ids)].copy()
icustays.intime = pd.to_datetime(icustays.intime)
icustays.outtime = pd.to_datetime(icustays.outtime)
icustays["ICU_LOS"] = (icustays.outtime - icustays.intime ).dt.days
icustays

,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los,ICU_LOS
37,402,302,191028,249750,carevue,MICU,MICU,15,15,2133-03-29 02:34:11,2133-03-30 22:50:35,1.8447,1
41,406,305,133059,232719,carevue,CCU,CCU,7,7,2125-04-27 18:47:43,2125-05-01 16:50:19,3.9185,3
42,407,305,108015,297009,carevue,CSRU,CSRU,15,15,2126-01-02 11:28:23,2126-01-07 10:07:29,4.9438,4
43,408,305,122211,224571,carevue,MICU,MICU,50,50,2127-06-19 23:50:45,2127-06-20 18:51:59,0.7925,0
44,409,305,122211,232248,carevue,MICU,MICU,50,50,2127-07-03 20:02:34,2127-07-09 15:31:36,5.8118,5
45,410,305,194340,217232,metavision,SICU,SICU,33,33,2129-09-03 12:31:31,2129-09-05 23:00:50,2.4370,2
50,415,309,162308,211035,carevue,CCU,CCU,7,7,2160-06-04 09:45:55,2160-06-05 19:03:42,1.3873,1
55,420,315,152144,266988,carevue,CSRU,CSRU,14,14,2177-04-29 08:04:53,2177-05-04 16:12:34,5.3387,5
58,423,318,193264,225173,carevue,CCU,CSRU,57,14,2129-06-07 08:50:00,2129-06-13 10:45:00,6.0799,6
60,425,320,172849,221205,carevue,CCU,CCU,57,57,2145-10-09 09:50:48,2145-10-09 20:39:48,0.4507,0


In [15]:
icu_lenOfStay = pd.DataFrame(icustays.groupby(["subject_id","hadm_id"]).ICU_LOS.sum())
icu_lenOfStay.reset_index(inplace=True)
df_ischemic = pd.merge(df_ischemic,icu_lenOfStay,on=["subject_id","hadm_id"],how='left')
df_ischemic.ICU_LOS.fillna(0,inplace=True)
df_ischemic

,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,diagnosis,row_id,gender,expire_flag,LENGTH_OF_STAY,AGE,AGE_BINS,LENGTH_OF_STAY_BINS,TOTAL_COST,PAYABLE,ICU_LOS
0,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,20.0,Male,0.0,5,71.0,"(70.2, 77.9]","(3.5, 7.0]",12468.0,1068.0,1.0
1,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,21.0,Male,0.0,2,39.0,"(31.7, 39.4]","(-0.035, 3.5]",6024.0,1024.0,0.0
2,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,25.0,Male,0.0,5,74.0,"(70.2, 77.9]","(3.5, 7.0]",52024.0,1024.0,1.0
3,35,166707,2122-02-10 11:15:00,2122-02-20 15:30:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,AORTIC VALVE DISEASE\CORONARY ARTERY BYPASS GR...,31.0,Female,0.0,10,69.0,"(62.5, 70.2]","(7.0, 10.5]",6024.0,1024.0,5.0
4,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,32.0,Male,0.0,8,69.0,"(62.5, 70.2]","(7.0, 10.5]",9824.0,1024.0,1.0
5,38,185910,2166-08-10 00:28:00,2166-09-04 11:30:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,ACUTE MYOCARDIAL INFARCTION-SEPSIS,34.0,Male,0.0,25,75.0,"(70.2, 77.9]","(24.5, 28.0]",30068.0,1068.0,25.0
6,359,144265,2176-06-30 03:00:00,2176-07-03 15:12:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Self Pay,VIET,BUDDHIST,MARRIED,OTHERS,R/O MYOCARDIAL INFARCTION,338.0,Male,0.0,3,48.0,"(47.1, 54.8]","(-0.035, 3.5]",5024.0,1024.0,3.0
7,362,142749,2112-07-10 02:31:00,2112-07-16 13:45:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,UNOBTAINABLE,WIDOWED,WHITE,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,341.0,Female,1.0,6,85.0,"(77.9, 85.6]","(3.5, 7.0]",4024.0,1024.0,0.0
8,365,139873,2147-12-15 17:10:00,2147-12-23 12:17:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,JEWISH,MARRIED,WHITE,CORONARY ARTERY DISEASE\CARDIAC CATH,344.0,Male,1.0,7,59.0,"(54.8, 62.5]","(3.5, 7.0]",1068.0,1068.0,1.0
9,44,181750,2192-11-19 18:14:00,2192-11-27 14:35:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,MARRIED,OTHERS,CORONARY ARTERY DISEASE\CATH,39.0,Male,1.0,7,80.0,"(77.9, 85.6]","(3.5, 7.0]",6068.0,1068.0,4.0


Using procedures performed on the patients as a feature:-

In [16]:
d_icd_procedures = ehr["d_icd_procedures"].copy()
d_icd_procedures.drop_duplicates(subset=["icd9_code"],keep='first',inplace=True)
d_icd_procedures

,row_id,icd9_code,short_title,long_title
0,264,851,Canthotomy,Canthotomy
1,265,852,Blepharorrhaphy,Blepharorrhaphy
2,266,859,Adjust lid position NEC,Other adjustment of lid position
3,267,861,Lid reconst w skin graft,Reconstruction of eyelid with skin flap or graft
4,268,862,Lid reconst w muc graft,Reconstruction of eyelid with mucous membrane ...
5,269,863,Lid reconst w hair graft,Reconstruction of eyelid with hair follicle graft
6,270,864,Lid recon-tarsoconj flap,Reconstruction of eyelid with tarsoconjunctiva...
7,271,869,Lid reconstr w graft NEC,Other reconstruction of eyelid with flaps or g...
8,272,870,Lid reconstruction NOS,"Reconstruction of eyelid, not otherwise specified"
9,273,871,Lid marg recon-part thic,"Reconstruction of eyelid involving lid margin,..."


In [17]:
procedures_icd = ehr["procedures_icd"]
procedures_icd

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331
5,949,55357,119355,3,3893
6,950,9545,158060,1,34
7,951,28600,189217,1,3613
8,952,28600,189217,2,3615
9,953,28600,189217,3,3961


In [18]:
proc = pd.merge(procedures_icd,d_icd_procedures,on=["icd9_code"],how='left')
proc[proc.subject_id.isin(sub_ids)]

,row_id_x,subject_id,hadm_id,seq_num,icd9_code,row_id_y,short_title,long_title
42,986,7671,160689,1,3949,748.0,Vasc proc revision NEC,Other revision of vascular procedure
43,987,7671,160689,2,3722,805.0,Left heart cardiac cath,Left heart cardiac catheterization
44,988,7671,160689,3,8853,3216.0,Lt heart angiocardiogram,Angiocardiography of left heart structures
45,989,7671,160689,4,8855,3218.0,Coronar arteriogr-1 cath,Coronary arteriography using a single catheter
156,1100,7546,181171,1,3895,803.0,Ven cath renal dialysis,Venous catheterization for renal dialysis
157,1101,7546,181171,2,3995,1423.0,Hemodialysis,Hemodialysis
158,1102,7546,181171,3,9672,3679.0,Cont inv mec ven 96+ hrs,Continuous invasive mechanical ventilation for...
159,1103,7546,181171,4,966,3676.0,Entral infus nutrit sub,Enteral infusion of concentrated nutritional s...
308,638,4113,146014,1,3613,663.0,Aortocor bypas-3 cor art,(Aorto)coronary bypass of three coronary arteries
309,639,4113,146014,2,3615,665.0,1 int mam-cor art bypass,Single internal mammary-coronary artery bypass


In [19]:
proc[proc.subject_id.isin(sub_ids)].icd9_code.value_counts()

3615    57
3961    55
8856    41
8853    38
3722    32
3612    29
3723    23
3601    20
9920    20
3606    16
3613    15
3611    11
8855    10
3761    10
9672     8
8872     6
3995     6
3404     5
3491     5
9604     5
3614     5
3605     5
9915     5
966      5
3521     4
4223     4
3794     4
8857     4
9671     3
3893     3
        ..
3844     1
3772     1
9390     1
8842     1
9925     1
8622     1
3512     1
8968     1
8964     1
4903     1
3721     1
7791     1
7781     1
8848     1
3616     1
3732     1
3845     1
3734     1
3949     1
3522     1
3783     1
5569     1
3963     1
4593     1
4579     1
8674     1
8669     1
9962     1
9961     1
3950     1
Name: icd9_code, Length: 95, dtype: int64

As we can see in the above cell, certain subject ids have undergone more than 20 procedures,while some just one. Having this as a feature might introduce too much noise

Incorporating prescriptions as a feature:-

In [20]:
prescriptions = ehr['prescriptions'][ehr['prescriptions'].subject_id.isin(sub_ids)].copy()
prescriptions

,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route
31,17267,239,127976,231723.0,2175-01-13,2175-01-13,MAIN,Furosemide,Furosemide,Furosemide,FURO40I,8205.0,4.096102e+08,40mg/4mL Vial,20,mg,0.5,VIAL,IV
32,17268,239,127976,231723.0,2175-01-13,2175-01-13,MAIN,Metoprolol,Metoprolol,Metoprolol,METO25,50631.0,5.107903e+10,25mg Tablet,25,mg,1,TAB,PO
33,17272,239,127976,231723.0,2175-01-13,2175-01-14,MAIN,Furosemide,Furosemide,Furosemide,FURO40I,8205.0,4.096102e+08,40mg/4mL Vial,20,mg,0.5,VIAL,IV
34,17452,239,127976,231723.0,2175-01-13,2175-01-14,MAIN,Cefazolin,NaN,NaN,CEFA10I,9066.0,6.332302e+10,10g Bulk vial,2,gm,0.2,VIAL,IV
35,17443,239,127976,231723.0,2175-01-13,2175-01-14,BASE,NS,NaN,NaN,NS100,1210.0,3.380049e+08,100mL Bag,100,ml,100,ml,IV
36,17413,239,127976,231723.0,2175-01-13,2175-01-14,MAIN,Metoprolol,Metoprolol,Metoprolol,METO5I,19808.0,5.539001e+10,5mg/5mL Vial,5,mg,1,VIAL,IV
37,17414,239,127976,231723.0,2175-01-13,2175-01-14,MAIN,Magnesium Sulfate,Magnesium Sulfate,Magnesium Sulfate,MAGS1I,1417.0,5.172602e+08,1g/2mL Vial,2,gm,4,ml,IV
38,17270,239,127976,231723.0,2175-01-13,2175-01-14,MAIN,Metoprolol,Metoprolol,Metoprolol,METO25,50631.0,5.107903e+10,25mg Tablet,25,mg,1,TAB,PO
39,17411,239,127976,231723.0,2175-01-13,2175-01-19,MAIN,Insulin,Insulin,Insulin - Sliding Scale,INSULIN,1723.0,0.000000e+00,Dummy Package for Sliding Scale,0,UNIT,0,VIAL,SC
40,17275,239,127976,231723.0,2175-01-13,2175-01-19,MAIN,Aspirin EC,Aspirin EC,Aspirin EC,ASA81EC,16995.0,9.047704e+08,81mg EC Tab,81,mg,1,TAB,PO


In [21]:
prescriptions.hadm_id.value_counts()

194340    208
122211    186
158975    182
152144    136
108015    104
133059     88
146586     79
101608     69
100095     64
116935     48
139873     47
109897     46
165617     45
192097     43
190642     38
121438     37
162308     37
135807     28
117941     26
191028     20
127976     18
190711     18
172849      8
167504      7
Name: hadm_id, dtype: int64

Very few subjects have take drugs and those who have taken,vary a lot. Adding each drug as a feature will just more noise

End of feature creation

# Final dataset:-

In [22]:
df_ischemic

,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,diagnosis,row_id,gender,expire_flag,LENGTH_OF_STAY,AGE,AGE_BINS,LENGTH_OF_STAY_BINS,TOTAL_COST,PAYABLE,ICU_LOS
0,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,20.0,Male,0.0,5,71.0,"(70.2, 77.9]","(3.5, 7.0]",12468.0,1068.0,1.0
1,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,21.0,Male,0.0,2,39.0,"(31.7, 39.4]","(-0.035, 3.5]",6024.0,1024.0,0.0
2,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,25.0,Male,0.0,5,74.0,"(70.2, 77.9]","(3.5, 7.0]",52024.0,1024.0,1.0
3,35,166707,2122-02-10 11:15:00,2122-02-20 15:30:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,AORTIC VALVE DISEASE\CORONARY ARTERY BYPASS GR...,31.0,Female,0.0,10,69.0,"(62.5, 70.2]","(7.0, 10.5]",6024.0,1024.0,5.0
4,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,32.0,Male,0.0,8,69.0,"(62.5, 70.2]","(7.0, 10.5]",9824.0,1024.0,1.0
5,38,185910,2166-08-10 00:28:00,2166-09-04 11:30:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,ACUTE MYOCARDIAL INFARCTION-SEPSIS,34.0,Male,0.0,25,75.0,"(70.2, 77.9]","(24.5, 28.0]",30068.0,1068.0,25.0
6,359,144265,2176-06-30 03:00:00,2176-07-03 15:12:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Self Pay,VIET,BUDDHIST,MARRIED,OTHERS,R/O MYOCARDIAL INFARCTION,338.0,Male,0.0,3,48.0,"(47.1, 54.8]","(-0.035, 3.5]",5024.0,1024.0,3.0
7,362,142749,2112-07-10 02:31:00,2112-07-16 13:45:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,UNOBTAINABLE,WIDOWED,WHITE,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,341.0,Female,1.0,6,85.0,"(77.9, 85.6]","(3.5, 7.0]",4024.0,1024.0,0.0
8,365,139873,2147-12-15 17:10:00,2147-12-23 12:17:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,JEWISH,MARRIED,WHITE,CORONARY ARTERY DISEASE\CARDIAC CATH,344.0,Male,1.0,7,59.0,"(54.8, 62.5]","(3.5, 7.0]",1068.0,1068.0,1.0
9,44,181750,2192-11-19 18:14:00,2192-11-27 14:35:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,MARRIED,OTHERS,CORONARY ARTERY DISEASE\CATH,39.0,Male,1.0,7,80.0,"(77.9, 85.6]","(3.5, 7.0]",6068.0,1068.0,4.0


Dropping several unnecessary columns:-

In [23]:
df_ischemic_final = df_ischemic.drop(columns=["subject_id","hadm_id","admittime","dischtime","diagnosis","row_id","expire_flag"])
del(df_ischemic)
df_ischemic_final

,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,gender,LENGTH_OF_STAY,AGE,AGE_BINS,LENGTH_OF_STAY_BINS,TOTAL_COST,PAYABLE,ICU_LOS
0,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,Male,5,71.0,"(70.2, 77.9]","(3.5, 7.0]",12468.0,1068.0,1.0
1,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,Male,2,39.0,"(31.7, 39.4]","(-0.035, 3.5]",6024.0,1024.0,0.0
2,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,Male,5,74.0,"(70.2, 77.9]","(3.5, 7.0]",52024.0,1024.0,1.0
3,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,Female,10,69.0,"(62.5, 70.2]","(7.0, 10.5]",6024.0,1024.0,5.0
4,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,Male,8,69.0,"(62.5, 70.2]","(7.0, 10.5]",9824.0,1024.0,1.0
5,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,Male,25,75.0,"(70.2, 77.9]","(24.5, 28.0]",30068.0,1068.0,25.0
6,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Self Pay,VIET,BUDDHIST,MARRIED,OTHERS,Male,3,48.0,"(47.1, 54.8]","(-0.035, 3.5]",5024.0,1024.0,3.0
7,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,UNOBTAINABLE,WIDOWED,WHITE,Female,6,85.0,"(77.9, 85.6]","(3.5, 7.0]",4024.0,1024.0,0.0
8,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,JEWISH,MARRIED,WHITE,Male,7,59.0,"(54.8, 62.5]","(3.5, 7.0]",1068.0,1068.0,1.0
9,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,MARRIED,OTHERS,Male,7,80.0,"(77.9, 85.6]","(3.5, 7.0]",6068.0,1068.0,4.0


Saving the dataset to use for building the predictive model

In [24]:
df_ischemic_final.to_csv("../Dataset/Final_ehr_claims/dataset.csv",index=False)